# Testing Dataset

Dependencies:

In [3]:
import os
import shutil
import random; random.seed(42)

import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import torch
from torch import nn
#import tensorflow as tf # tensorflow-gpu==2.0.0
#from tensorflow.python.client import device_lib 
#print(device_lib.list_local_devices())
#import cv2

Directories:

In [4]:
# training data
train_dir = 'Data/train_and_test_split/dpc_dataset_traintest_4_200_csv/train'
train_dir_video = 'Data/train_and_test_split/dpc_dataset_traintest_4_200_h264/train'

# test data
test_inputs_dir = 'Data/train_and_test_split/dpc_dataset_traintest_4_200_csv/test_inputs/'
test_targets_dir = 'Data/train_and_test_split/dpc_dataset_traintest_4_200_csv/test_targets/'
test_targets_video = 'Data/train_and_test_split/dpc_dataset_traintest_4_200_h264/test_targets/'

# validation data
validation_inputs_dir = 'Data/train_and_test_split/dpc_dataset_traintest_4_200_csv/validation_inputs/'
validation_targets_dir = 'tData/rain_and_test_split/dpc_dataset_traintest_4_200_csv/validation_targets/'
validation_targets_video = 'Data/train_and_test_split/dpc_dataset_traintest_4_200_h264/validation_targets/'

## Coordinate based predictions:
### Data Transformation Functions

In [5]:
# some constants
DEFAULT_X_RED, DEFAULT_Y_RED = (240, 240)

PIXEL_DISTANCE_GREEN_TO_RED = 118 # approx. value | calculated with the Pythagorean theorem and averaged: np.sqrt((y_green-y_red)**2 + (x_green-x_red)**2)
PIXEL_DISTANCE_BLUE_TO_GREEN = 90 # approx. value | calculated with the Pythagorean theorem and averaged: np.sqrt((y_blue-y_green)**2 + (x_blue-x_green)**2)

def raw_to_pixel(l):
    '''Convert the raw coordinates to pixel coordinates.'''
    assert isinstance(l, list)
    return [x/5 for x in l]


def pixel_to_raw(l):
    '''Convert the pixel coordinates to raw coordinates.'''
    assert isinstance(l, list)
    return [x*5 for x in l]


def raw_cartesian_to_polar_angles(l):
    '''Convert the cartesian coordinates to polar coordinates.'''
    assert isinstance(l, list)
    x_red, y_red, x_green, y_green, x_blue, y_blue = raw_to_pixel(l)

    angle_green_red = np.arctan((y_green-y_red)/(x_green-x_red+0.001))
    angle_blue_green = np.arctan((y_blue-y_green)/(x_blue-x_green+0.001))
    
    return [np.sin(angle_green_red), np.cos(angle_green_red), np.sin(angle_blue_green), np.cos(angle_blue_green)]

def polar_angles_to_raw_cartesian(l):
    '''Convert the polar coordinates back to cartesian coordinates.'''
    assert isinstance(l, list)
    sin_angle_green_red, cos_angle_green_red, sin_angle_blue_green, cos_angle_blue_green = l
    
    y_green = PIXEL_DISTANCE_GREEN_TO_RED * sin_angle_green_red + DEFAULT_Y_RED
    x_green = PIXEL_DISTANCE_GREEN_TO_RED * cos_angle_green_red + DEFAULT_X_RED

    y_blue = PIXEL_DISTANCE_BLUE_TO_GREEN * sin_angle_blue_green + y_green
    x_blue = PIXEL_DISTANCE_BLUE_TO_GREEN * cos_angle_blue_green + x_green
    
    return pixel_to_raw([DEFAULT_X_RED, DEFAULT_Y_RED, x_green, y_green, x_blue, y_blue])

Verify that the raw -> pixel conversion and pixel -> raw works as intended, and that the cartesian -> polar conversion and polar -> cartesian conversion works as intended.

In [6]:
raw_coordinates = list(np.array([240, 240, 357.4438349670886, 228.55685234634907, 444.41827493559794, 205.41712909467287])*5)
pixel_coordinates = raw_to_pixel(raw_coordinates)
new_raw_coordinates = pixel_to_raw(pixel_coordinates)
assert raw_coordinates == new_raw_coordinates, '`Raw -> Pixel` and `Pixel -> Raw` coordinate conversion methods are malfunctioning.'

raw_cartesian = list(np.array([240, 240, 357.4438349670886, 228.55685234634907, 444.41827493559794, 205.41712909467287])*5)
polar = raw_cartesian_to_polar_angles(raw_cartesian)
new_raw_cartesian = polar_angles_to_raw_cartesian(polar)
assert [round(x) for x in raw_cartesian] == [round(x) for x in new_raw_cartesian], 'Cartesian to Polar and Polar to Cartesian methods are malfunctioning.'

Data reading functions

Parsing training data:
training data x-y matching is like this:
x: a list of 4 frames
y: the frame that follows

In [7]:
def parse_training_annotations(csv_file):
    '''Parse the training annotations from a CSV file.'''
    X_data = []
    y_data = []
    f = pd.read_csv(csv_file, header=None, delim_whitespace=True, engine='python')
    temp = []
    for i, row in f.iterrows():
        if len(temp) < 4:
            # convert the cartesian pixel coordinates to polar coordinates
            temp.append(raw_cartesian_to_polar_angles(row.to_list()))
        else:
            # the output frame
            # convert the cartesian pixel coordinates to polar coordinates
            next_frame = raw_cartesian_to_polar_angles(row.to_list())

            # save
            X_data.append(temp.copy())
            y_data.append(next_frame.copy())

            # add output frame to the inputs and remove the first
            temp.pop(0)
            temp.append(next_frame)
    return X_data, y_data

Load in data

In [8]:
BATCH_SIZE = 4000

# load in all separate files
X = []
y = []
for filename in tqdm([x for x in os.listdir(train_dir) if not x.startswith('.')]):
    # load in a file
    X_data, y_data = parse_training_annotations(os.path.join(train_dir, filename))

    X = X + X_data
    y = y + y_data

100%|██████████| 40/40 [01:00<00:00,  1.51s/it]


In [17]:
class DoublePendulumDataset(torch.utils.data.Dataset):
    def __init__(self,X_list,y_list):
        self.sample_list = list(zip(X_list, y_list))
    
    def __getitem__(self,index):
        X_sample,y_sample = self.sample_list[index]
        return X_sample,y_sample
    
    def __len__(self):
        return len(self.sample_list)

In [19]:
myDataSet = DoublePendulumDataset(X,y)
myDataLoader = torch.utils.data.DataLoader(myDataSet,batch_size=4000)

In [20]:
class Model(nn.Module):
    def __init__(self):
        # We want a model of 4 layer LSTM with 32 features output, and a dense layer to form the 4 feature output.
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_size = 32
        self.n_layers = 4

        #Defining the layers
        # LSTM layer
        self.lstm = nn.LSTM(input_size = 4, hidden_size = 32, num_layers = 4, batch_first = True)

        # Fully connected layer
        self.fc = nn.Linear(32, 4)
    
    def forward(self, x):
        #h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_size).requires_grad_()
        #c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_size).requires_grad_()
        out= self.lstm(x) # (h0.detach(), c0.detach())
        out = out[:, -1, :]
        out = self.fc(out)

        return out

In [21]:
class Optimization:
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []
    
    def train_step(self, x, y):
        # Sets model to train mode
        self.model.train()

        # Makes predictions
        yhat = self.model(x)

        # Computes loss
        loss = self.loss_fn(y, yhat)

        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss
        return loss.item()

In [22]:
# Instantiate the model with hyperparameters
model = Model()

# Define hyperparameters
n_epochs = 10
lr=0.01

# Define Loss, Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [28]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(myDataLoader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(torch.Tensor(inputs))
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

ValueError: only one element tensors can be converted to Python scalars

In [14]:

for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    output = model(torch_X)
    loss = criterion(output.view(-1), torch_y.view(-1))
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%1 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

NameError: name 'torch_X' is not defined

In [26]:
def plot_history(history):
    """Plot the training process of a model."""
    
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch') 
    plt.ylabel('Mean Squared Error [MSE]')
    plt.plot(hist['epoch'], hist['mean_squared_error'].tolist(),
           label='Train Error')
    plt.legend()

In [18]:
def parse_testing_input(csv_file):
    '''Parse the training annotations from a CSV file.'''
    X_data = []
    f = pd.read_csv(csv_file, header=None, delim_whitespace=True, engine='python')
    temp = []
    for i, row in f.iterrows():
            temp.append(raw_cartesian_to_polar_angles(row.to_list()))
    X_data.append(temp.copy())
    return X_data

def parse_testing_target(csv_file):
    '''Parse the training annotations from a CSV file.'''
    Y_data = []
    f = pd.read_csv(csv_file, header=None, delim_whitespace=True, engine='python')
    for i, row in f.iterrows():
        Y_data.append(raw_cartesian_to_polar_angles(row.to_list()))
        break
    return Y_data

# load in all separate files
X_test = []
y_test = []
for filename in tqdm([x for x in os.listdir(test_inputs_dir) if not x.startswith('.')]):
    # load in a file
    X_data_test=parse_testing_input(os.path.join(test_inputs_dir, filename))
    y_data_test=parse_testing_target(os.path.join(test_targets_dir, filename))
    
    X_test.append(X_data_test)
    y_test.append(y_data_test)

num_batches = len(X_test)
num_records = num_batches * BATCH_SIZE
print(f'{num_records} training records spread over {num_batches} batches of size {BATCH_SIZE}')

100%|██████████| 60/60 [00:00<00:00, 164.45it/s]

240000 training records spread over 60 batches of size 4000


In [19]:
X_test = np.array(X_test)
y_test = np.array(y_test)
# torch_X has n batches of size 4000, each sample is 4 frame, each frame is 4 value (sin,cos,sin,cos), e.g. torch.Size([69,4000,4,4])
torch_X_test = torch.from_numpy(X_test)
# torch_y has n batches of size 4000, each sample is a sequence of frames, unknown length, of 4 values, e.g. torch.Size([69,4000,4])
torch_y_test = torch.from_numpy(y_test)
torch_X_test = torch_X_test.view(-1,4,4).float()
torch_y_test = torch_y_test.view(-1,4).float()

In [20]:
test_output = model(torch_X_test)

In [28]:
torch_y_test

tensor([[-1.0000,  0.0034,  0.5655,  0.8248],
        [-0.3178,  0.9482, -0.8746,  0.4849],
        [-0.3807,  0.9247,  0.8750,  0.4841],
        [ 0.7917,  0.6109,  0.4247,  0.9053],
        [-0.9556,  0.2948,  0.4437,  0.8962],
        [ 0.7754,  0.6315,  0.5439,  0.8392],
        [-0.9744,  0.2249, -0.8289,  0.5593],
        [-0.0356,  0.9994, -0.6706,  0.7419],
        [-0.8248,  0.5654,  0.6440,  0.7650],
        [ 0.9940,  0.1091,  0.4647,  0.8854],
        [-0.2819,  0.9594,  0.1268,  0.9919],
        [-0.7264,  0.6873,  0.9998,  0.0177],
        [ 0.5000,  0.8660, -0.9518,  0.3069],
        [-0.3801,  0.9249,  0.2331,  0.9724],
        [-0.9950,  0.1004, -0.9136,  0.4066],
        [-0.0492,  0.9988, -0.3036,  0.9528],
        [ 0.2725,  0.9622, -0.8770,  0.4806],
        [ 0.9936,  0.1127,  0.2890,  0.9573],
        [ 0.0915,  0.9958, -0.3649,  0.9310],
        [-0.3807,  0.9247, -0.6947,  0.7193],
        [-0.5027,  0.8645,  0.0777,  0.9970],
        [-0.9351,  0.3543, -0.9243

In [30]:
test_output

tensor([[ 0.1009,  0.5713, -0.0491,  0.5025],
        [ 0.1049,  0.5731, -0.0550,  0.5052],
        [ 0.1076,  0.5541, -0.0351,  0.4866],
        [ 0.1159,  0.5373, -0.0329,  0.4772],
        [ 0.1013,  0.5712, -0.0485,  0.5021],
        [ 0.1157,  0.5392, -0.0337,  0.4785],
        [ 0.1002,  0.5793, -0.0578,  0.5103],
        [ 0.1084,  0.5672, -0.0519,  0.5006],
        [ 0.1039,  0.5657, -0.0434,  0.4966],
        [ 0.1160,  0.5352, -0.0344,  0.4772],
        [ 0.1060,  0.5688, -0.0477,  0.4997],
        [ 0.1026,  0.5730, -0.0533,  0.5047],
        [ 0.1109,  0.5583, -0.0496,  0.4951],
        [ 0.1057,  0.5690, -0.0477,  0.4998],
        [ 0.1000,  0.5779, -0.0575,  0.5095],
        [ 0.1076,  0.5689, -0.0512,  0.5012],
        [ 0.1098,  0.5629, -0.0509,  0.4979],
        [ 0.1162,  0.5333, -0.0330,  0.4757],
        [ 0.1080,  0.5683, -0.0514,  0.5009],
        [ 0.1040,  0.5764, -0.0556,  0.5071],
        [ 0.1040,  0.5748, -0.0524,  0.5050],
        [ 0.1006,  0.5798, -0.0578

In [21]:
diff = (torch_y_test-test_output) / torch_y_test

In [24]:
diff.double()

tensor([[ 1.1009e+00, -1.6596e+02,  1.0868e+00,  3.9077e-01],
        [ 1.3300e+00,  3.9554e-01,  9.3708e-01, -4.1950e-02],
        [ 1.2826e+00,  4.0072e-01,  1.0401e+00, -5.0896e-03],
        [ 8.5356e-01,  1.2045e-01,  1.0775e+00,  4.7289e-01],
        [ 1.1060e+00, -9.3762e-01,  1.1094e+00,  4.3976e-01],
        [ 8.5080e-01,  1.4618e-01,  1.0620e+00,  4.2984e-01],
        [ 1.1029e+00, -1.5763e+00,  9.3023e-01,  8.7726e-02],
        [ 4.0410e+00,  4.3242e-01,  9.2260e-01,  3.2521e-01],
        [ 1.1260e+00, -4.8345e-04,  1.0674e+00,  3.5090e-01],
        [ 8.8330e-01, -3.9047e+00,  1.0740e+00,  4.6107e-01],
        [ 1.3759e+00,  4.0721e-01,  1.3763e+00,  4.9626e-01],
        [ 1.1412e+00,  1.6636e-01,  1.0533e+00, -2.7539e+01],
        [ 7.7815e-01,  3.5534e-01,  9.4793e-01, -6.1347e-01],
        [ 1.2781e+00,  3.8479e-01,  1.2046e+00,  4.8599e-01],
        [ 1.1005e+00, -4.7587e+00,  9.3702e-01, -2.5326e-01],
        [ 3.1886e+00,  4.3037e-01,  8.3122e-01,  4.7401e-01],
        

In [27]:
plot_history(test_output)

AttributeError: 'Tensor' object has no attribute 'history'